In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Pará - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [5]:
data = pd.read_csv('2003_mo_model_input_PA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - Desemprego,Pará - Produção de Cimento (t),Pará - value,Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Pará - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.348543,49.618560,0.312867,0.669564,7.776091e+07,4.250430e+06,9.220374,6.851269e+07,84.701
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.342554,49.658098,0.312657,0.669764,7.787181e+07,4.255476e+06,9.225493,6.856365e+07,80.221
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.336566,49.906876,0.311736,0.669963,7.798271e+07,4.260521e+06,9.230612,6.861461e+07,76.122
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.330578,49.864305,0.310493,0.670163,7.809361e+07,4.265566e+06,9.235731,6.866557e+07,76.544
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.324589,50.179753,0.309128,0.670362,7.820451e+07,4.270611e+06,9.240851,6.871652e+07,83.445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.899018,0.710173,NaN,NaN,NaN,NaN,NaN,156.442
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.416567,0.708384,NaN,NaN,NaN,NaN,NaN,145.777
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.559167,0.706038,NaN,NaN,NaN,NaN,NaN,139.233
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.696005,0.702147,NaN,NaN,NaN,NaN,NaN,142.025


In [6]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - Desemprego,Pará - Produção de Cimento (t),Pará - value,Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.743008,-1.860097,-1.001954,-1.906833,-1.652592,-1.810463,-2.287151,-1.950096
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.748918,-1.858558,-1.004769,-1.880912,-1.635387,-1.758101,-2.245800,-1.920309
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.754829,-1.848873,-1.017098,-1.854991,-1.618182,-1.705740,-2.204450,-1.890522
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.760740,-1.850531,-1.033739,-1.829071,-1.600977,-1.653378,-2.163099,-1.860734
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.766650,-1.838250,-1.052007,-1.803150,-1.583772,-1.601017,-2.121748,-1.830947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.076246,-0.644020,1.421704,1.389768,1.316401,-0.980846,1.292832,1.315283
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.073514,-0.676398,1.447588,1.378488,1.314484,-0.973932,1.302757,1.318820
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.070782,-0.703221,1.485640,1.367208,1.312566,-0.967018,1.312682,1.322356
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.068050,-0.725198,1.528780,1.355927,1.310648,-0.960105,1.322607,1.325893


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      110.204
1       76.354
2       77.171
3       80.720
4       82.088
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Pará - Consumo de Cimento (t), Length: 240, dtype: float64

In [8]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - Desemprego,Pará - Produção de Cimento (t),Pará - value,Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.743008,-1.860097,-1.001954,-1.906833,-1.652592,-1.810463,-2.287151,-1.950096
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.748918,-1.858558,-1.004769,-1.880912,-1.635387,-1.758101,-2.245800,-1.920309
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.754829,-1.848873,-1.017098,-1.854991,-1.618182,-1.705740,-2.204450,-1.890522
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.760740,-1.850531,-1.033739,-1.829071,-1.600977,-1.653378,-2.163099,-1.860734
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.766650,-1.838250,-1.052007,-1.803150,-1.583772,-1.601017,-2.121748,-1.830947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.444243,0.582557,1.197817,1.368533,1.186547,-1.094003,0.861099,1.054711
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.421118,0.538378,1.208551,1.376125,1.197116,-1.099823,0.885880,1.071769
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.397993,0.482165,1.218732,1.383716,1.207686,-1.105643,0.910662,1.088827
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.374869,0.427013,1.224310,1.391308,1.218256,-1.111463,0.935444,1.105885


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      110.204
1       76.354
2       77.171
3       80.720
4       82.088
        ...   
157    116.872
158    131.959
159    116.171
160    132.242
161    135.615
Name: Pará - Consumo de Cimento (t), Length: 162, dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - Desemprego,Pará - Produção de Cimento (t),Pará - value,Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
162,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,1.328619,0.370821,1.234144,1.406491,1.239395,-1.123103,0.985007,1.140002
163,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,1.305494,0.346293,1.237610,1.414083,1.249965,-1.128924,1.009789,1.157060
164,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,1.282370,0.310674,1.240190,1.421675,1.260535,-1.134744,1.034570,1.174119
165,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,1.259245,0.268772,1.243109,1.429266,1.271104,-1.140564,1.059352,1.191177
166,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,1.236120,0.241945,1.248836,1.436858,1.281674,-1.146384,1.084133,1.208235
167,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,1.212995,0.204282,1.255236,1.444450,1.292244,-1.152204,1.108915,1.225293
168,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,1.189871,0.153365,1.262362,1.452041,1.302813,-1.158024,1.133697,1.242352
169,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,1.181995,0.093487,1.269260,1.453432,1.305064,-1.147293,1.141168,1.246366
170,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,1.174120,0.053981,1.275633,1.454823,1.307315,-1.136561,1.148640,1.250381
171,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,1.166245,-0.005914,1.283830,1.456214,1.309566,-1.125829,1.156111,1.254395


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    136.117
163    136.475
164    125.300
165    132.627
166    135.172
167    131.376
168    111.175
169     80.970
170     92.173
171     86.782
172     85.150
173    135.094
174    132.351
175    133.744
176    128.527
177    147.510
178    138.511
179    103.294
180    110.704
181     91.638
182     90.744
183     82.108
184     97.490
185     96.803
186    118.687
187    109.761
188    120.188
189    131.620
190    125.374
191    115.811
Name: Pará - Consumo de Cimento (t), dtype: float64

In [12]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [13]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
#     train, train_val = validation_splitter(train_input, 7)
#     target,target_val = validation_splitter(train_target, 7)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
#                         validation_data=(train_val, target_val),
                        validation_split=0.15,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [14]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [16]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1155073950, 4127564519, 4268157112, 117630895, 1220293349, 3729691422, 3079667129, 2892921510, 462838517, 2166315401, 2965600535, 3815600089, 3742299691, 2704448227, 2879542555, 3418611701, 3605175257, 4208621217, 397666552, 620985116, 3711763314, 3033530022, 2361560174, 3248782365, 1901269241]


Step: 0 ___________________________________________
val_loss: 1955.41455078125
winner_seed: 1155073950


Step: 1 ___________________________________________
val_loss: 10839.0849609375


Step: 2 ___________________________________________
val_loss: 2412.80615234375


Step: 3 ___________________________________________
val_loss: 31371.859375


Step: 4 ___________________________________________
val_loss: 9074.78125


Step: 5 ___________________________________________
val_loss: 3226.8525390625


Step: 6 ___________________________________________
val_loss: 14695.2646484375


Step: 7 ___________________________________________
val_loss: 22060.560546875


Step: 8 _________________________________

In [17]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 29ms/step - loss: 10711.1455 - val_loss: 82792.1875
Epoch 2/10000
5/5 [==============================] - 0s 8ms/step - loss: 1584.2185 - val_loss: 84119.1797
Epoch 3/10000
5/5 [==============================] - 0s 7ms/step - loss: 1567.0201 - val_loss: 55324.5703
Epoch 4/10000
5/5 [==============================] - 0s 6ms/step - loss: 1906.1250 - val_loss: 71747.7500
Epoch 5/10000
5/5 [==============================] - 0s 7ms/step - loss: 1899.1161 - val_loss: 57912.4883
Epoch 6/10000
5/5 [==============================] - 0s 7ms/step - loss: 1726.6151 - val_loss: 69884.1250
Epoch 7/10000
5/5 [==============================] - 0s 7ms/step - loss: 1594.6587 - val_loss: 66386.9766
Epoch 8/10000
5/5 [==============================] - 0s 7ms/step - loss: 1692.6511 - val_loss: 80306.2891
Epoch 9/10000
5/5 [==============================] - 0s 7ms/step - loss: 1581.9858 - val_loss: 69296.8516
Epoch 10/10000
5/5 [========================

5/5 [==============================] - 0s 7ms/step - loss: 315.6865 - val_loss: 6858.1831
Epoch 79/10000
5/5 [==============================] - 0s 7ms/step - loss: 343.2788 - val_loss: 8469.0723
Epoch 80/10000
5/5 [==============================] - 0s 7ms/step - loss: 303.4919 - val_loss: 7004.6636
Epoch 81/10000
5/5 [==============================] - 0s 7ms/step - loss: 334.0952 - val_loss: 9528.3916
Epoch 82/10000
5/5 [==============================] - 0s 7ms/step - loss: 342.8581 - val_loss: 8593.6582
Epoch 83/10000
5/5 [==============================] - 0s 7ms/step - loss: 308.8289 - val_loss: 8297.3945
Epoch 84/10000
5/5 [==============================] - 0s 8ms/step - loss: 363.0735 - val_loss: 8517.2061
Epoch 85/10000
5/5 [==============================] - 0s 7ms/step - loss: 349.7541 - val_loss: 8382.2422
Epoch 86/10000
5/5 [==============================] - 0s 8ms/step - loss: 297.7665 - val_loss: 7370.2451
Epoch 87/10000
5/5 [==============================] - 0s 7ms/step - lo

5/5 [==============================] - 0s 7ms/step - loss: 298.2831 - val_loss: 8825.1123
Epoch 156/10000
5/5 [==============================] - 0s 8ms/step - loss: 297.0966 - val_loss: 8242.3291
Epoch 157/10000
5/5 [==============================] - 0s 7ms/step - loss: 306.2920 - val_loss: 8585.8467
Epoch 158/10000
5/5 [==============================] - 0s 7ms/step - loss: 313.8726 - val_loss: 9330.0479
Epoch 159/10000
5/5 [==============================] - 0s 6ms/step - loss: 342.8815 - val_loss: 8339.4209
Epoch 160/10000
5/5 [==============================] - 0s 7ms/step - loss: 302.4640 - val_loss: 7394.9941
Epoch 161/10000
5/5 [==============================] - 0s 7ms/step - loss: 341.0638 - val_loss: 7582.2012
Epoch 162/10000
5/5 [==============================] - 0s 7ms/step - loss: 324.0218 - val_loss: 7745.9448
Epoch 163/10000
5/5 [==============================] - 0s 7ms/step - loss: 325.2747 - val_loss: 7715.1401
Epoch 164/10000
5/5 [==============================] - 0s 7ms/

5/5 [==============================] - 0s 7ms/step - loss: 397.2440 - val_loss: 8420.8623
Epoch 233/10000
5/5 [==============================] - 0s 7ms/step - loss: 355.7939 - val_loss: 8461.5684
Epoch 234/10000
5/5 [==============================] - 0s 7ms/step - loss: 298.0836 - val_loss: 7660.8877
Epoch 235/10000
5/5 [==============================] - 0s 7ms/step - loss: 300.5868 - val_loss: 8575.5264
Epoch 236/10000
5/5 [==============================] - 0s 7ms/step - loss: 312.8229 - val_loss: 9377.2383
Epoch 237/10000
5/5 [==============================] - 0s 7ms/step - loss: 334.2924 - val_loss: 8884.2002
Epoch 238/10000
5/5 [==============================] - 0s 7ms/step - loss: 379.6753 - val_loss: 9082.3750
Epoch 239/10000
5/5 [==============================] - 0s 7ms/step - loss: 370.4004 - val_loss: 7837.2061
Epoch 240/10000
5/5 [==============================] - 0s 7ms/step - loss: 357.6869 - val_loss: 8205.6797
Epoch 241/10000
5/5 [==============================] - 0s 7ms/

5/5 [==============================] - 0s 7ms/step - loss: 350.7906 - val_loss: 8886.5869
Epoch 310/10000
5/5 [==============================] - 0s 7ms/step - loss: 346.1696 - val_loss: 8203.6221
Epoch 311/10000
5/5 [==============================] - 0s 7ms/step - loss: 350.8658 - val_loss: 7986.3511
Epoch 312/10000
5/5 [==============================] - 0s 8ms/step - loss: 351.7170 - val_loss: 9563.0664
Epoch 313/10000
5/5 [==============================] - 0s 7ms/step - loss: 345.7433 - val_loss: 9526.3877
Epoch 314/10000
5/5 [==============================] - 0s 7ms/step - loss: 315.6797 - val_loss: 7168.6924
Epoch 315/10000
5/5 [==============================] - 0s 7ms/step - loss: 321.9199 - val_loss: 8401.0010
Epoch 316/10000
5/5 [==============================] - 0s 7ms/step - loss: 314.8066 - val_loss: 8702.9238
Epoch 317/10000
5/5 [==============================] - 0s 7ms/step - loss: 312.4934 - val_loss: 8993.2061
Epoch 318/10000
5/5 [==============================] - 0s 7ms/

...

5/5 [==============================] - 0s 7ms/step - loss: 512.7880 - val_loss: 973.4838
Epoch 1157/10000
5/5 [==============================] - 0s 7ms/step - loss: 432.2341 - val_loss: 667.6066
Epoch 1158/10000
5/5 [==============================] - 0s 7ms/step - loss: 455.9838 - val_loss: 1033.6279
Epoch 1159/10000
5/5 [==============================] - 0s 7ms/step - loss: 434.8080 - val_loss: 629.2502
Epoch 1160/10000
5/5 [==============================] - 0s 6ms/step - loss: 460.4828 - val_loss: 730.0522
Epoch 1161/10000
5/5 [==============================] - 0s 7ms/step - loss: 419.6341 - val_loss: 730.4241
Epoch 1162/10000
5/5 [==============================] - 0s 6ms/step - loss: 450.5179 - val_loss: 680.1808
Epoch 1163/10000
5/5 [==============================] - 0s 7ms/step - loss: 442.6258 - val_loss: 793.8692
Epoch 1164/10000
5/5 [==============================] - 0s 7ms/step - loss: 435.3235 - val_loss: 746.2216
Epoch 1165/10000
5/5 [==============================] - 0s 6ms

5/5 [==============================] - 0s 7ms/step - loss: 469.7631 - val_loss: 854.6557
Epoch 1234/10000
5/5 [==============================] - 0s 6ms/step - loss: 456.0487 - val_loss: 924.0858
Epoch 1235/10000
5/5 [==============================] - 0s 6ms/step - loss: 456.7849 - val_loss: 1185.7231
Epoch 1236/10000
5/5 [==============================] - 0s 12ms/step - loss: 462.3079 - val_loss: 1298.7593
Epoch 1237/10000
5/5 [==============================] - 0s 7ms/step - loss: 501.2220 - val_loss: 755.4523
Epoch 1238/10000
5/5 [==============================] - 0s 8ms/step - loss: 420.8266 - val_loss: 696.2249
Epoch 1239/10000
5/5 [==============================] - 0s 7ms/step - loss: 434.9320 - val_loss: 661.6417
Epoch 1240/10000
5/5 [==============================] - 0s 7ms/step - loss: 421.7355 - val_loss: 736.7517
Epoch 1241/10000
5/5 [==============================] - 0s 7ms/step - loss: 427.4352 - val_loss: 784.2788
Epoch 1242/10000
5/5 [==============================] - 0s 6

5/5 [==============================] - 0s 7ms/step - loss: 432.4263 - val_loss: 1351.3110
Epoch 1311/10000
5/5 [==============================] - 0s 8ms/step - loss: 471.8499 - val_loss: 594.5599
Epoch 1312/10000
5/5 [==============================] - 0s 7ms/step - loss: 454.7906 - val_loss: 1142.1399
Epoch 1313/10000
5/5 [==============================] - 0s 7ms/step - loss: 446.3460 - val_loss: 1149.8009
Epoch 1314/10000
5/5 [==============================] - 0s 7ms/step - loss: 434.0632 - val_loss: 1096.0242
Epoch 1315/10000
5/5 [==============================] - 0s 6ms/step - loss: 459.3667 - val_loss: 683.9114
Epoch 1316/10000
5/5 [==============================] - 0s 7ms/step - loss: 500.1150 - val_loss: 662.6536
Epoch 1317/10000
5/5 [==============================] - 0s 7ms/step - loss: 432.5886 - val_loss: 589.9456
Epoch 1318/10000
5/5 [==============================] - 0s 7ms/step - loss: 431.6314 - val_loss: 1008.4866
Epoch 1319/10000
5/5 [==============================] - 0s

5/5 [==============================] - 0s 7ms/step - loss: 452.7160 - val_loss: 1102.6199
Epoch 1388/10000
5/5 [==============================] - 0s 7ms/step - loss: 432.7774 - val_loss: 865.7682
Epoch 1389/10000
5/5 [==============================] - 0s 7ms/step - loss: 424.8063 - val_loss: 826.8000
Epoch 1390/10000
5/5 [==============================] - 0s 8ms/step - loss: 429.9261 - val_loss: 834.8052
Epoch 1391/10000
5/5 [==============================] - 0s 8ms/step - loss: 425.1179 - val_loss: 852.6620
Epoch 1392/10000
5/5 [==============================] - 0s 7ms/step - loss: 428.1357 - val_loss: 1001.6395
Epoch 1393/10000
5/5 [==============================] - 0s 8ms/step - loss: 410.2788 - val_loss: 859.7748
Epoch 1394/10000
5/5 [==============================] - 0s 6ms/step - loss: 497.1768 - val_loss: 579.8337
Epoch 1395/10000
5/5 [==============================] - 0s 7ms/step - loss: 471.5342 - val_loss: 1429.9398
Epoch 1396/10000
5/5 [==============================] - 0s 7

5/5 [==============================] - 0s 7ms/step - loss: 454.3669 - val_loss: 550.5276
Epoch 1465/10000
5/5 [==============================] - 0s 7ms/step - loss: 517.9698 - val_loss: 1106.8094
Epoch 1466/10000
5/5 [==============================] - 0s 6ms/step - loss: 424.6389 - val_loss: 648.9114
Epoch 1467/10000
5/5 [==============================] - 0s 7ms/step - loss: 446.8245 - val_loss: 581.2504
Epoch 1468/10000
5/5 [==============================] - 0s 7ms/step - loss: 464.1501 - val_loss: 915.3735
Epoch 1469/10000
5/5 [==============================] - 0s 7ms/step - loss: 549.7877 - val_loss: 640.5441
Epoch 1470/10000
5/5 [==============================] - 0s 7ms/step - loss: 435.1791 - val_loss: 643.1362
Epoch 1471/10000
5/5 [==============================] - 0s 7ms/step - loss: 428.5913 - val_loss: 797.2225
Epoch 1472/10000
5/5 [==============================] - 0s 7ms/step - loss: 414.7534 - val_loss: 697.0345
Epoch 1473/10000
5/5 [==============================] - 0s 7ms

In [18]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [19]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 16ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699,141.241699
Target,136.117,136.475,125.3,132.627,135.172,131.376,111.175,80.97,92.173,86.782,85.15,135.094,132.351,133.744,128.527,147.51,138.511,103.294,110.704,91.638,90.744,82.108,97.49,96.803,118.687,109.761,120.188,131.62,125.374,115.811
Error,5.124695,4.766693,15.941696,8.6147,6.069702,9.865692,30.066696,60.271698,49.068703,54.459702,56.091698,6.147705,8.890701,7.497696,12.714706,6.268295,2.730698,37.947701,30.537697,49.603699,50.497696,59.133698,43.751701,44.438698,22.554703,31.480698,21.053696,9.621704,15.867699,25.430702


In [20]:
display(mae)
display(mape)

26.217052

0.26628044

In [21]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [22]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 18ms/step
Ano-0: |Prediction[[1694.9004]] - Target[1388.411]| =  Error: [[306.48938]]; MAPE:[[0.2207483]]
1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[1694.9004]] - Target[1353.424]| =  Error: [[341.47644]]; MAPE:[[0.2523056]]
1/1 [==============================] - 0s 16ms/step
Ano-5: |Prediction[[847.4502]] - Target[721.441]| =  Error: [[126.00922]]; MAPE:[[0.17466323]]


[array([[306.48938]], dtype=float32),
 array([[341.47644]], dtype=float32),
 array([[126.00922]], dtype=float32)]

257.99167

0.21590571